In [2]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

'Connected: None@factbook.db'

# Analyzing CIA Factbook data using SQL

The following notebook looks at the CIA's Factbook dataset and uses SQL to analyze some basic questions about popuation size, area, birth rate, death rate, and migration rate. 

> The World Factbook provides information on the history, people and society, government, economy, energy, geography, communications, transportation, military, and transnational issues for 267 world entities. 

Here is a sample of the dataset that's being analyzed, but you can find more information about the dataset [here](https://www.cia.gov/library/publications/the-world-factbook/).




In [29]:
%%sql
SELECT *
FROM facts 
LIMIT 5

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


### What are the minimum and maximum amounts for population and population growth? 

In [9]:
%%sql
SELECT 
    MIN(population) as min_population,
    MAX(population) as max_population,
    MIN(population_growth) as min_pop_percent,
    MAX(population_growth) as max_pop_percent
FROM facts 

Done.


min_population,max_population,min_pop_percent,max_pop_percent
0,7256490011,0.0,4.02


### Identifying outliers 

When looking at the dataset, the minimum and maximum numbers don't look quite right. Taking a closer look at the names of these countries in the cells below they are for "World" and "Antarctica" and those will need to be removed when answers any other questions.


In [30]:
%%sql

SELECT 
    name,
    population
FROM
    facts 
WHERE population = (SELECT MIN(population) from facts)

Done.


name,population
Antarctica,0


In [31]:
%%sql

SELECT 
    name,
    population
FROM 
    facts 
WHERE population = (SELECT MAX(population) from facts)

Done.


name,population
World,7256490011


### Summary statistics with outliers removed 

In [26]:
%%sql

SELECT 
    MIN(population) as min_pop,
    MAX(population) as max_pop,
    MIN(population_growth) as min_pop_percent,
    MAX(population_growth) as max_pop_percent
FROM
    facts 
WHERE name NOT IN ("World", "Antarctica")


Done.


min_pop,max_pop,min_pop_percent,max_pop_percent
48,1367485388,0.0,4.02


### Average population and area 

In [28]:
%%sql

SELECT 
    ROUND(AVG(population),2) as avg_population,
    ROUND(AVG(area),2) as avg_area
FROM
    facts

Done.


avg_population,avg_area
62094928.32,555093.55


## Which countries are densely populated?

The following query looks at countries with an above average population size **and** below average area to see which countries are densely populated compared to their size. 

_Area is the combined land and water mass of a given country_


In [32]:
%%sql

SELECT 
    name,
    population,
    area
FROM
    facts 
WHERE 
    population > (SELECT AVG(population) from facts)
AND 
    area < (SELECT AVG(area) from facts)

Done.


name,population,area
Bangladesh,168957745,148460
Germany,80854408,357022
Japan,126919659,377915
Philippines,100998376,300000
Thailand,67976405,513120
United Kingdom,64088222,243610
Vietnam,94348835,331210


### What countries have the most people? What countries have the highest growth rates?

In [57]:
%%sql

SELECT 
    name,
    population,
    population_growth
FROM
    facts
WHERE name NOT IN ("World", "European Union") 
ORDER BY 2 DESC
LIMIT 10 


Done.


name,population,population_growth
China,1367485388,0.45
India,1251695584,1.22
United States,321368864,0.78
Indonesia,255993674,0.92
Brazil,204259812,0.77
Pakistan,199085847,1.46
Nigeria,181562056,2.45
Bangladesh,168957745,1.6
Russia,142423773,0.04
Japan,126919659,0.16


In [56]:
%%sql 

SELECT 
    name,
    population,
    population_growth
FROM
    facts
WHERE name NOT IN ("World", "European Union") 
ORDER BY 3 DESC
LIMIT 10 


Done.


name,population,population_growth
South Sudan,12042910,4.02
Malawi,17964697,3.32
Burundi,10742276,3.28
Niger,18045729,3.25
Uganda,37101745,3.24
Qatar,2194817,3.07
Burkina Faso,18931686,3.03
Mali,16955536,2.98
Cook Islands,9838,2.95
Iraq,37056169,2.93


### ### Which countries will add the most people to their populations next year?

In [ ]:
%%sql 



### Which countries have the highest ratios of water to land? 

In [60]:
%%sql

SELECT name, 
       ROUND(CAST(area_water AS float) / CAST(area_land AS float), 2) AS water_to_land_ratio
FROM facts
WHERE name NOT IN ('British Indian Ocean Territory')
ORDER BY 2 DESC
LIMIT 10

Done.


name,water_to_land_ratio
Virgin Islands,4.52
Puerto Rico,0.55
"Bahamas, The",0.39
Guinea-Bissau,0.28
Malawi,0.26
Netherlands,0.23
Uganda,0.22
Eritrea,0.16
Liberia,0.16
Bangladesh,0.14


### What countries have more water than land?

In [68]:
%%sql

SELECT 
    name,
    area_water,
    area_land
FROM
    facts
WHERE
    area_water > area_land
AND
    name NOT IN ("British Indian Ocean Territory")


Done.


name,area_water,area_land
Virgin Islands,1564,346


### How many countries have a higher death rate than birth rate? List the by death rate. 

In [72]:
%%sql

SELECT 
    COUNT(*) as total_countries
FROM 
    facts 
WHERE 
    death_rate > birth_rate



Done.


total_countries
24


In [77]:
%%sql
SELECT 
    name,
    death_rate,
    birth_rate
FROM
    facts
WHERE
    death_rate > birth_rate
ORDER BY 2 DESC

Done.


name,death_rate,birth_rate
Ukraine,14.46,10.72
Bulgaria,14.44,8.92
Latvia,14.31,10.0
Lithuania,14.27,10.1
Russia,13.69,11.6
Serbia,13.66,9.08
Belarus,13.36,10.7
Hungary,12.73,9.16
Moldova,12.59,12.0
Estonia,12.4,10.51


### Which countries have the highest population/area ratio, and how does it compare to list we found in the previous screen?

In [81]:
%%sql

SELECT 
    name,
    population,
    area,
    (population / area)  as pop_density
FROM
    facts 
ORDER BY 4 DESC
LIMIT 10


Done.


name,population,area,pop_density
Macau,592731,28,21168
Monaco,30535,2,15267
Singapore,5674472,697,8141
Hong Kong,7141106,1108,6445
Gaza Strip,1869055,360,5191
Gibraltar,29258,6,4876
Bahrain,1346613,760,1771
Maldives,393253,298,1319
Malta,413965,316,1310
Bermuda,70196,54,1299


The countries that have the highest population density are different than the previously pulled data for countries with above average population and below average area. That is because these counties, although they have a below average area mass, do not have higher than average population when compared to counties like China, India, and the United States. 